In [1]:
import numpy as np
import pandas as pd
import urllib.request
from PIL import Image
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score, mean_absolute_error
import math
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
diamonds = pd.read_csv("diamonds_train.csv", index_col=0)

In [3]:
#Poner una categoria de mayor valor al clarity
diamonds_df = diamonds.copy()
diamonds_df["clarity_value"] = diamonds_df["clarity"].map({'I1': 1, 'SI2': 2, 'SI1': 3, 'VS2':4, 'VS1':5, 'VVS2':6, 'VVS1':7, 'IF':8 })

In [4]:
#Poner una categoria de mayor valor al color
diamonds_df["color_value"] = diamonds_df["color"].map({'J': 1, 'I': 2, 'H': 3, 'G':4, 'F':5, 'E':6, 'D':7})

In [5]:
#Poner una categoria de mayor valor al corte
diamonds_df["cut_value"] = diamonds_df["cut"].map({'Fair': 1, 'Good': 2, 'Very Good': 3, 'Premium':4, 'Ideal':5})
diamonds_df

,carat,cut,color,clarity,depth,table,x,y,z,price,clarity_value,color_value,cut_value
0,1.21,Ideal,H,VS2,63.0,57.0,6.73,6.70,4.23,6134,4,3,5
1,0.28,Very Good,D,VVS2,64.0,56.0,4.14,4.17,2.66,532,6,7,3
2,0.42,Premium,F,VS1,61.2,58.0,4.86,4.82,2.96,1103,5,5,4
3,0.26,Ideal,H,IF,61.1,57.0,4.16,4.12,2.53,600,8,3,5
4,1.10,Good,G,SI1,63.4,57.0,6.52,6.55,4.14,4997,3,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40340,1.55,Premium,H,VS2,61.3,61.0,7.46,7.39,4.55,11708,4,3,4
40341,0.36,Ideal,D,SI1,60.6,56.0,4.58,4.63,2.79,619,3,7,5
40342,0.57,Very Good,I,VS2,62.2,55.0,5.33,5.34,3.32,1267,4,2,3
40343,1.01,Very Good,F,IF,59.6,62.0,6.47,6.56,3.88,9965,8,5,3


## 1. Definir X e y

In [18]:
#X = np.array(diamonds_df[['x','y','z', 'clarity_value', 'color_value', 'cut_value' ]])
#y = np.array(diamonds_df['price'])

In [6]:
X = np.array(diamonds_df[['carat', 'depth', 'table','x','y','z', 'clarity_value', 'color_value', 'cut_value' ]])
y = np.array(diamonds_df['price'])

In [7]:
X = np.array(diamonds_df[['carat','x','y','z', 'clarity_value', 'color_value', 'cut_value' ]])
y = np.array(diamonds_df['price'])

In [7]:
# el test_size 0.20 y el random state puede cambiar 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [8]:
# Prueba regresion no lineal con polinomio
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_absolute_error
degree = 3
polinominal_model = PolynomialFeatures(degree) 
X_poly = polinominal_model.fit_transform(X_train, y_train)



In [9]:
lin_reg_model = LinearRegression()
lin_reg_model.fit(X_poly, y_train)
y_pred = lin_reg_model.predict(X_poly)

NameError: name 'X_poly' is not defined

In [15]:
np.sqrt(mean_squared_error(y_train, y_pred))

600.5987720737977

In [8]:
import numpy as np
from catboost import Pool, CatBoostRegressor

# initialize Pool
train_pool = Pool(X_train, 
                  y_train, 
                  cat_features=[0,2,5])
test_pool = Pool(X_test, 
                 cat_features=[0,2,5])
model = CatBoostRegressor(iterations=2,
                          learning_rate=1,
                          depth=2)

CatBoostError: 'data' is numpy array of floating point numerical type, it means no categorical features, but 'cat_features' parameter specifies nonzero number of categorical features

In [12]:
model.fit(train_pool)

0:	learn: 1863.7330597	total: 151ms	remaining: 151ms
1:	learn: 1689.3355806	total: 156ms	remaining: 0us


In [13]:
predictions = model.predict(test_pool)
print(predictions)

[ 6202.44447122 14877.79420921  1827.62365646 ...  6202.44447122
   789.37515273   789.37515273]


In [14]:
np.sqrt(mean_squared_error(y_test, predictions))

1689.3882684336381

In [9]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
model = RandomForestRegressor(n_estimators = 500, max_features= 'auto', max_depth= 15, min_samples_split= 5, min_samples_leaf= 2, n_jobs = 1, verbose=2, random_state =42)

In [9]:
model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
building tree 1 of 500
building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building

RandomForestRegressor(max_depth=15, min_samples_leaf=2, min_samples_split=5,
                      n_estimators=500, n_jobs=1, random_state=42, verbose=2)

In [10]:
y_pred = model.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.7s finished


In [11]:
rmse = mean_squared_error(y_test, y_pred)**0.5

In [12]:
np.sqrt(mean_squared_error(y_test, y_pred))

552.785124391953

In [11]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [8]:

model = RandomForestRegressor(n_estimators =800, max_features= 'auto', max_depth= 90, min_samples_split= 4, min_samples_leaf= 10, n_jobs = 1, verbose=2, random_state =1)


model.fit(X_train, y_train)

| elapsed:    0.0s remaining:    0.0s
building tree 1 of 800
building tree 2 of 800
building tree 3 of 800
building tree 4 of 800
building tree 5 of 800
building tree 6 of 800
building tree 7 of 800
building tree 8 of 800
building tree 9 of 800
building tree 10 of 800
building tree 11 of 800
building tree 12 of 800
building tree 13 of 800
building tree 14 of 800
building tree 15 of 800
building tree 16 of 800
building tree 17 of 800
building tree 18 of 800
building tree 19 of 800
building tree 20 of 800
building tree 21 of 800
building tree 22 of 800
building tree 23 of 800
building tree 24 of 800
building tree 25 of 800
building tree 26 of 800
building tree 27 of 800
building tree 28 of 800
building tree 29 of 800
building tree 30 of 800
building tree 31 of 800
building tree 32 of 800
building tree 33 of 800
building tree 34 of 800
building tree 35 of 800
building tree 36 of 800
building tree 37 of 800
building tree 38 of 800
building tree 39 of 800
building tree 40 of 800
building tr

RandomForestRegressor(max_depth=90, min_samples_leaf=10, min_samples_split=4,
                      n_estimators=800, n_jobs=1, random_state=1, verbose=2)

In [9]:
predictions = model.predict(X_test)
print(predictions)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[5766.57289549 3492.23027861  626.08984431 ...  510.41967067 5219.95525668
 2280.09345532]
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    1.4s finished


In [10]:
np.sqrt(mean_squared_error(y_test, predictions))

562.5838856264456

In [ ]:
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(forest, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(x_train, y_train)

In [10]:
model.fit(X_train,y_train)

#y_pred=model.predict(X_test)

RandomForestRegressor(n_estimators=20, random_state=0)

In [11]:
predictions = model.predict(X_test)
print(predictions)

[ 6450.5  15228.35  2402.05 ...  3592.    1163.     761.7 ]


In [12]:
np.sqrt(mean_squared_error(y_test, predictions))

564.0698003308596

In [ ]:
rmse = mean_squared_error(y_test, y_pred)**0.5

In [ ]:
print('RMSE   : %0.2f ' % rmse)

In [ ]:
## OTRA OPCION

In [9]:
rfr = RandomForestRegressor()

grid_param = {'n_estimators': [500, 800], 
              'max_features': ('auto', 'sqrt', 'log2'),
                'max_depth': [10, 20],
                'min_samples_split': [2, 5],
                'min_samples_leaf': [1, 2] }


modelo = GridSearchCV(estimator = rfr,
                  param_grid = grid_param,
                  n_jobs=-1,
                  cv=10,
                  verbose=2)

modelo.fit(X_train, y_train)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


In [8]:
model = RandomForestRegressor(n_estimators =100, max_features= 'auto', max_depth= 25, min_samples_split= 10, min_samples_leaf= 5, n_jobs = 1, verbose=2, random_state =1)

In [9]:
model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building

RandomForestRegressor(max_depth=25, min_samples_leaf=5, min_samples_split=10,
                      n_jobs=1, random_state=1, verbose=2)

In [10]:
 y_pred = model.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [11]:
rmse = mean_squared_error(y_test, y_pred)**0.5

In [12]:
np.sqrt(mean_squared_error(y_test, y_pred))

546.6634616396793

In [49]:
X_pred = pd.read_csv("diamonds_test.csv", index_col=0)

In [50]:
X_pred = X_pred.copy()
X_pred["clarity_value"] = X_pred["clarity"].map({'I1': 1, 'SI2': 2, 'SI1': 3, 'VS2':4, 'VS1':5, 'VVS2':6, 'VVS1':7, 'IF':8 })

In [51]:
X_pred["cut_value"] = X_pred["cut"].map({'Fair': 1, 'Good': 2, 'Very Good': 3, 'Premium':4, 'Ideal':5})

In [53]:
X_pred = np.array(X_pred[['carat','x','y','z', 'clarity_value', 'color_value', 'cut_value']])

In [52]:
X_pred["color_value"] = X_pred["color"].map({'J': 1, 'I': 2, 'H': 3, 'G':4, 'F':5, 'E':6, 'D':7})
X_pred

,carat,cut,color,clarity,depth,table,x,y,z,clarity_value,cut_value,color_value
0,0.30,Ideal,H,SI2,60.0,56.0,4.41,4.43,2.65,2,5,3
1,0.34,Ideal,D,IF,62.1,57.0,4.52,4.46,2.79,8,5,7
2,1.57,Very Good,I,VS2,60.3,58.0,7.58,7.55,4.56,4,3,2
3,0.31,Ideal,H,VS2,61.8,57.0,4.32,4.36,2.68,4,5,3
4,1.51,Good,I,VVS1,64.0,60.0,7.26,7.21,4.63,7,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
13444,1.05,Ideal,F,SI2,60.3,55.0,6.55,6.68,3.99,2,5,5
13445,0.27,Very Good,G,VS1,61.9,56.0,4.13,4.16,2.56,5,3,4
13446,1.50,Premium,D,VS2,61.8,60.0,7.30,7.37,4.53,4,4,7
13447,1.70,Premium,F,SI2,60.2,61.0,7.79,7.68,4.66,2,4,5


In [54]:
sample = pd.read_csv("sample_submission.csv")

In [57]:
predictions_submit = model.predict(X_pred)
predictions_submit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    1.6s finished


array([  422.39700289,  1750.4842002 ,  9698.55743413, ...,
       15108.52918648, 10460.43590729,   722.37541496])

In [58]:
submission = pd.DataFrame({"id": range(len(predictions_submit)), "price": predictions_submit})

In [59]:
submission.head()

,id,price
0,0,422.397003
1,1,1750.484200
2,2,9698.557434
3,3,505.184113
4,4,8914.201043


In [60]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")

In [ ]:
#number of trees
n_estimators = [500, 800, 1500, 2500, 5000]
#max number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2' ]
#max number of levels in tree 
max_depth = [10, 20, 30, 40, 50]
max_depth.append(None)
#Minimum number of samples to split a model 
min_samples_split = [2, 5, 10, 15, 20]
#Minimum number of samples required at each node
min_samples_leaf = [1, 2, 5, 10, 15]


grid_param = {'n_estimators':n_estimators, 
              'max_features':max_features,
                'max_depth':max_depth,
                'min_samples_split':min_samples_split,
                'min_samples_leaf':min_samples_leaf}


In [61]:
chequeator(submission)

You're ready to submit!


In [ ]:
# en este caso solo damos opciones del parámetro kernel, aunque tenemos C, gamma, degree, epsilon, coef0, entre otros 
kernel = ['rbf', 'linear', 'poly']

#lista de columnas numéricas, sin Price
col_num = df._get_numeric_data().columns[:-1]

dicti_svr_scores = {"column":[], "kernel":[], "score":[], "standarize":[]}

for col in col_num:
    for ker in kernel:
        for i in range(2):
            print("################")
            X = np.array(df[col]).reshape(-1, 1)
            y = np.array(df.Price).reshape(-1, 1)
            
            standard = False
            if i == 1:
                sc_X = StandardScaler()
                sc_y = StandardScaler()
                X = sc_X.fit_transform(X.reshape(-1, 1))
                y = sc_y.fit_transform(y.reshape(-1, 1))
                standard = True 

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

            #######
            print(X_train.shape)
            y_train = y_train.ravel()
            print(y_train.shape)

            model = SVR(kernel=ker)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_train)

            score = r2_score(y_train, y_pred)

            print("y_pred", y_pred.shape)
            print("score:", score)
            
            #código para crear el dataframe de score            
            lista_val = [col, ker, score, standard]
            for i,e in enumerate(["column", "kernel", "score", "standarize"]):
                dicti_svr_scores[e].append(lista_val[i])

In [ ]:
df_svr_score = pd.DataFrame(dicti_svr_scores)

In [ ]:
df_svr_score_grouped = df_svr_score.groupby(['column', 'kernel','standarize']).max().unstack()

In [ ]:
df_svr_score_grouped.columns = ['Not standarized', 'Standarized']

In [ ]:
df_svr_score_grouped

In [18]:
#number of trees
n_estimators = [500, 800, 1500, 2500, 5000]
#max number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2' ]
#max number of levels in tree 
max_depth = [10, 20, 30, 40, 50]
max_depth.append(None)
#Minimum number of samples to split a model 
min_samples_split = [2, 5, 10, 15, 20]
#Minimum number of samples required at each node
min_samples_leaf = [1, 2, 5, 10, 15]


grid_param = {'n_estimators':n_estimators, 
              'max_features':max_features,
                'max_depth':max_depth,
                'min_samples_split':min_samples_split,
                'min_samples_leaf':min_samples_leaf}



In [19]:
from sklearn.model_selection import RandomizedSearchCV
RFR=RandomForestRegressor(random_state=1)
RFR_random = RandomizedSearchCV(estimator= RFR,
            param_distributions= grid_param, n_iter=500,
            cv=5, verbose=2, random_state=42, n_jobs= -1 )

In [20]:
RFR_random.fit(X_train, y_train)
print(RFR_random.best_params_)

NameError: name 'x_train' is not defined

In [ ]:
print('GridSearch CV best score : {:.4f}\n\n'.format(grid_search.best_score_))

print('Parameters that give the best results :','\n\n', (grid_search.best_params_))

print('\n\nEstimator that was chosen by the search :','\n\n', (grid_search.best_estimator_))

In [ ]:
rmse = mean_squared_error(y_test, y_pred)**0.5

In [ ]:
print('GridSearch CV score on test set: {0:0.4f}'.format(grid_search.score(X_test_grid, y_test_grid)))

In [ ]:
print('RMSE   : %0.2f ' % rmse)